In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt

import os

print(os.getcwd()) # dosya yolunu ver
%run ../Model.ipynb
%run ../Dataset.ipynb

plt.rcParams['font.size'] = 20
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']

/arf/home/tunal/ondemand/PhD Thesis Starting/01_SON/Tik-4/Tez/01-Moon/02-UAE_for_Moon


In [2]:
class Trainer:
    def __init__(self, model, optimizer, device='cpu', max_patience=20):
        self.model = model.to(device)
        self.optimizer = optimizer
        self.device = device
        
        self.max_patience = max_patience
        self.best_val_loss = float('inf')
        self.patience = 0

        self.val_cost = []
        self.train_cost = []
    
    def train(self, train_loader, val_loader=None, epochs=10, print_every=1, name=None):
        self.model.train()
        
        for epoch in range(1, epochs + 1):
            total_loss = 0.0
            for x, x_, _ in train_loader:  # label kullanılmıyor
                x = x.to(self.device)
                x_ = x_.to(self.device)

                with torch.no_grad():
                    z_hat_ = self.model.encoder(x_)
                z_hat, x_hat = self.model(x)
                
                loss = self.model.criterion(z_pred=z_hat_, z_true=z_hat, x_pred=x_hat, x_true=x.reshape(-1,2))

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                total_loss += loss.item()

            # Validation (opsiyonel)
            if val_loader:
                val_loss = self.validate(val_loader)
                
                # Early stopping kontrolü
                if val_loss < self.best_val_loss:
                    print('saved!')
                    torch.save(self.model, name + '.model')
                    self.best_val_loss = val_loss
                    self.patience = 0
        
                else:
                    self.patience = self.patience + 1
        
                if self.patience > self.max_patience:
                    break
                    
            if epoch % print_every == 0:
                print(f"Epoch {epoch:3d} | Train Loss: {total_loss / len(train_loader):.6f} | Validation Loss: {val_loss:.6f}")

            self.val_cost.append(val_loss)
            self.train_cost.append(total_loss / len(train_loader))
            
    def validate(self, val_loader):
        self.model.eval()
        total_loss = 0.0
        
        with torch.no_grad():
            for x, x_, _ in val_loader:
                x = x.to(self.device)
                x_ = x_.to(self.device)
                
                z_hat_ = self.model.encoder(x_)
                z_hat, x_hat = self.model(x)
                
                loss = self.model.criterion(z_pred=z_hat_, z_true=z_hat, x_pred=x_hat, x_true=x.reshape(-1,2))
                
                total_loss += loss.item()
                
        avg_loss = total_loss / len(val_loader)
        #print(f"→ Validation Loss: {avg_loss:.6f}")
        self.model.train()
        return avg_loss

In [3]:
# Custom Transform
class NoiseTransform:
    """Add some noise."""

    def __init__(self, split_ratio=0.001, dim=2):

        self.normal_dist = split_ratio*np.random.randn(dim,)
        
    def __call__(self, x):
      return x + self.normal_dist

In [4]:
# Hyper-Parameters & Settings

dataset_size = 1000
batch_size = 500
lr = 0.003

epochs = 1000
max_patience = 1000

split_ratio = 0.0001

In [5]:
# Dataset
train_dataset = MoonDataset(mode='train', n_samples=dataset_size, transform=NoiseTransform(split_ratio))
val_dataset = MoonDataset(mode='val', n_samples=dataset_size, transform=NoiseTransform(split_ratio))
test_dataset = MoonDataset(mode='test', n_samples=dataset_size)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=True)

In [6]:
# "results" klasörünü oluştur (zaten varsa hata vermez)
os.makedirs("results", exist_ok=True)

In [7]:
# Model
name = 'results/UAE_Moon'

model = To_Uniform(
                 input_dim=2,
                 latent_dim=1,
                 output_dim=2,
                 encoder_hidden=64,
                 decoder_hidden=64,
                 encoder_act=nn.SiLU,
                 decoder_act=nn.SiLU,
                 final_encoder_act=nn.Sigmoid,
                 final_decoder_act=nn.Sigmoid
)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [8]:
# Training
trainer = Trainer(model, optimizer, device='cuda' if torch.cuda.is_available() else 'cpu', max_patience=max_patience)
trainer.train(train_loader, val_loader, epochs=epochs, name=name)

saved!
Epoch   1 | Train Loss: 0.051078 | Validation Loss: 0.059366
saved!
Epoch   2 | Train Loss: 0.047724 | Validation Loss: 0.057508
Epoch   3 | Train Loss: 0.045106 | Validation Loss: 0.057808
saved!
Epoch   4 | Train Loss: 0.041582 | Validation Loss: 0.057149
saved!
Epoch   5 | Train Loss: 0.038984 | Validation Loss: 0.052496
saved!
Epoch   6 | Train Loss: 0.033478 | Validation Loss: 0.043985
saved!
Epoch   7 | Train Loss: 0.027888 | Validation Loss: 0.033794
saved!
Epoch   8 | Train Loss: 0.021733 | Validation Loss: 0.023639
saved!
Epoch   9 | Train Loss: 0.017512 | Validation Loss: 0.018411
Epoch  10 | Train Loss: 0.016387 | Validation Loss: 0.018657
Epoch  11 | Train Loss: 0.016498 | Validation Loss: 0.018697
saved!
Epoch  12 | Train Loss: 0.015521 | Validation Loss: 0.015951
saved!
Epoch  13 | Train Loss: 0.013241 | Validation Loss: 0.013766
Epoch  14 | Train Loss: 0.012110 | Validation Loss: 0.015366
Epoch  15 | Train Loss: 0.011253 | Validation Loss: 0.015148
Epoch  16 | Tra

In [ ]:
# CSV dosyasına kaydet
train_losses = trainer.train_cost  # liste veya numpy array
val_losses = trainer.val_cost

np.savetxt("results/losses.csv", 
           np.column_stack((train_losses, val_losses)), 
           delimiter=",", 
           header="train_loss,val_loss", 
           comments="")